In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import os
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot  as plt
from hydra.utils import instantiate

In [6]:
df_label = pd.read_csv("../data/data.csv").set_index("cif.label")

In [26]:
df_vector = pd.read_csv("../data/embedding_example/embedding-h50rl5oi.csv")

In [21]:
df_vector[df_vector["type"] == "RSM0001"].values[0][1:]

array([0.18223792, -0.052778434, -0.025544835, 0.06903792, 0.029065704,
       -0.09969002, -0.05961714, 0.045185998, -0.009621638, -0.018773982,
       -0.1577318, 0.012376932, 0.028340044, 0.12140944, -0.009764762,
       -0.07460559, -0.0028007084, 0.141223, 0.008262484, 0.03491501,
       0.028810509, 0.036006797, 0.0981388, 0.05978615, -0.087274544,
       -0.051497117, 0.11102457, -0.028097415, -0.15029188, -0.12889598,
       0.06776593, -0.0054096878, 0.0934243, 0.09503806, 0.03360904,
       -0.014801894, 0.01870705, 0.15593277, -0.030861687, -0.13042733,
       -0.1098428, -0.025176018, 0.015759083, -0.02503141, -0.004129014,
       0.04422977, 0.03908542, 0.018760912, 0.025280785, 0.14403401,
       0.01516605, 0.10004629, 0.11729335, 0.07898504, -0.0619319,
       0.03696615, 0.058077104, 0.06493654, -0.0036908805, -0.104176216,
       -0.053878434, 0.05886356, 0.003976745, 0.08749568], dtype=object)

In [24]:
embedded_mofs = list(df_vector["type"])

In [27]:
df = df_label[df_label["cif.label"].isin(embedded_mofs)].set_index("cif.label")

In [29]:
df.loc[embedded_mofs]

,Unnamed: 0,in,ASA [m^2/cm^3],CellV [A^3],Df,Di,Dif,NASA [m^2/cm^3],POAV [cm^3/g],POAVF,...,CH4DC,CH4HPSTP,CH4LPSTP,KH_CH4_molkgpa,KH_N2_molkgpa,log_selectivity,info.decorated_graph_hash,info.undecorated_graph_hash,info.decorated_scaffold_hash,info.undecorated_scaffold_hash
cif.label,,,,,,,,,,,,,,,,,,,,,
RSM0001,1514,3369,0.00,509.836,2.90906,3.78929,3.78929,4.28485,0.000000,0.00000,...,97.797569,106.963405,9.165836,9.192540e-14,4.295700e-12,NaN,397d5168298ea48e35e50c2dd7a5545a,d4cebadc19022cd2a9e55a4495c5acef,81ea15d99b10284b531d774c56dbfee8,aa1bb49eb59ff97e8ad55a10f3dd9621
RSM0002,1512,3367,0.00,516.368,2.92117,3.81858,3.81858,12.18450,0.000000,0.00000,...,121.197291,133.183754,11.986463,6.626080e-10,5.763760e-09,NaN,a4e6d5c237a4b8d50ee22509fdde0f71,8eaa459531a4cf79a0a7fd63471c3dc8,b37e56e8e46c3037ff39b1e8de78021a,9f71e84043e0476025e94bcb16499a31
RSM0003,1515,3370,0.00,505.611,2.91404,3.79670,3.79670,3.97516,0.000000,0.00000,...,94.161336,103.160279,8.998943,1.743300e-12,2.316240e-11,NaN,322048a06288c5929e6ea286dac69437,a1555cf381d317884a0699ab7e235bff,a8b8eb5294eec3d775f514d3fd72aa19,c1db4da8b3ce2177f74deed2ce155f69
RSM0004,1513,3368,0.00,513.366,2.90303,3.79357,3.79357,4.25538,0.000000,0.00000,...,105.647526,115.032205,9.384679,1.865900e-43,1.449720e-30,NaN,846aaf0e61abf76908df2b9692a36c5a,c06db605793b7b30ecf2808d05a3dff6,af7ea2651880fc8b73d5e0e430757672,f8a07a173c429da735b29411d328cb98
RSM0008,1914,4200,0.00,402.319,3.25805,4.50899,4.50899,352.76300,0.000000,0.00000,...,48.780682,151.280604,102.499922,1.148170e-05,3.701850e-06,NaN,fea58a3d5fbd7cf4c45abb387db33e9c,d6981923f6c12b6a0709175972451a27,d60fdebe602bdd993c7035c6c8281a55,2149ecdd31248123650b3512f93983a6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RSM4610,3653,8068,1348.40,34210.600,10.38181,17.46562,17.46523,89.26680,0.633173,0.49770,...,140.130858,206.113002,65.982143,9.340340e-06,2.239260e-06,0.620099,8d4876d71fafeab2e9950513d7cc0876,c060d1706f3242f9c376e8fcd6129f99,d00035e491c0d50d9777411a2971e47e,40f9f42cf2c0e310990337de642828c3
RSM4613,730,1631,1296.54,26193.900,5.46518,12.53445,12.53445,0.00000,0.362413,0.36704,...,95.189356,168.763739,73.574383,1.177000e-05,2.620370e-06,0.653581,7db8e54f19d42b74d1abbfd2d91accbd,a9d16815ce5e20294cf97df486ab367e,d097c1cee7b5f486aa55ea0e6028e75c,f5b96573524885e4ee1b98d555c6e0b3
RSM4614,939,2047,2137.73,47281.200,6.35591,10.98199,10.98199,0.00000,0.932101,0.61572,...,177.342684,271.618123,94.275439,1.428870e-05,4.426650e-06,0.508932,d31c95971a4de2dc413da1d256afe34b,efac990af7c16fe0a57f1792a7429652,1364dd3fdef45d84aaab7bca5998dc3a,dc9c6868d3cbbeddc8a703a9c97064f8


In [32]:
train_valid_idx, test_idx = train_test_split(range(len(df)), test_size=0.1)

In [36]:
len(train_valid_idx), len(test_idx)

(2931, 326)

In [37]:
train_idx, valid_idx = train_test_split(train_valid_idx, test_size=0.1)

In [39]:
len(train_idx), len(valid_idx)

(2637, 294)

In [41]:
train_names = [embedded_mofs[i] for i in train_idx]

In [47]:
df_label.set_index("cif.label").loc[train_names]["logKH_CO2"].values

array([-4.52613656, -4.54110241, -4.65836964, ..., -1.15562224,
       -3.36397774, -2.13238875])

In [18]:
from mofgraph2vec.model.vecnn import VecModel
from mofgraph2vec.model.nn_lightning import VecLightningModule

In [19]:
vec_model = VecModel(
    input_dim=53,
    output_dim=1,
    fcnn_n_layers=5,
    fcnn_hidden_size=128,
    fcnn_activation="relu",
)

In [27]:
my_model = VecLightningModule(model=vec_model, loss="mse", lr=1e-4)

In [35]:
from omegaconf import OmegaConf

In [36]:
model_conf = OmegaConf.load("../conf/model/nn.yaml")

In [39]:
model = instantiate(model_conf.nn)

In [41]:
pl_model = VecLightningModule(model, "mse, 1e-4")

In [42]:
pl_model

VecLightningModule(
  (model): VecModel(
    (embedding): Linear(in_features=53, out_features=128, bias=True)
    (convs): ModuleList(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): Linear(in_features=128, out_features=128, bias=True)
      (4): Linear(in_features=128, out_features=128, bias=True)
    )
    (activation): ReLU()
    (output): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [43]:
pl_model.load_from_checkpoint("../experiments/supervised/lightning_logs/version_0/checkpoints/epoch=99-step=4200.ckpt")

TypeError: __init__() missing 1 required positional argument: 'model'

In [29]:
my_model.model

VecModel(
  (embedding): Linear(in_features=53, out_features=128, bias=True)
  (convs): ModuleList(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Linear(in_features=128, out_features=128, bias=True)
  )
  (activation): ReLU()
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [5]:
state_dict = torch.load("../experiments/supervised/lightning_logs/version_0/checkpoints/epoch=99-step=4200.ckpt")

In [8]:
state_dict["state_dict"]['model.embedding.weight'].shape

torch.Size([128, 53])